In [1]:
import pandas as pd
import numpy as np

# read data from Aurora/Redshift (for writing may be some already made utilies in db_tools)
from db_tools.data_manager import sql_to_pandas, sql_to_pandas_big
from db_tools.request_data import RequestData
from db_tools.object_repository import ObjectRepository

from common.logger import logger
from common.data_utils.df_utils import sql_list_to_tuple

from python_engine.pipelines.pipeline_repository import PipelineRepository, BUCKET
from python_engine.data.sales import SalesRepository
from python_engine.data.category import CategoryRepository
from python_engine.data.store import StoreRepository
from python_engine.data.product import ProductRepository
from python_engine.data.stocks import StocksRepository
from python_engine.modules.features.transformations.apply_rolling_mean import apply_rolling_mean
from python_engine.modules.features.product.repository import ProductFeaturesRepository
from python_engine.modules.features.transformations.fill_dates import fill_dates
from python_engine.modules.common.dataframes.cartesian_merge import merge_cartesian

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

import datetime

import matplotlib.pyplot as plt
# import seaborn as sns
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import json

# read/write data in S3

# tables/dataframes
# https://github.com/nextail/py-db-tools/blob/master/db_tools/s3_utils.py
# Note 1: also the DataFrame/parquet version
# Note 2: also some utilities with the url version (not needed to pass the bucket). If you want write the rest and do a PR! :)
from db_tools.s3_utils import save_df_to_csv_in_s3, read_df_from_csv_in_s3, read_df_from_csv_in_s3_url

# any file (JSON for example)
from db_tools.object_repository import ObjectRepository
from common.logger import logger

# you may want to use another searilirez so you don't depend on python-engine
# from python_engine.modules.common.util.json_utils import serialize_to_str

In [2]:
client = 'pimkie'
environment = 'sandbox'
request_data = RequestData(client=client, environment=environment)
request_data.aurora_con, request_data.redshift_con

(<sqlalchemy.engine.base.Connection at 0x7f4af23f7e48>,
 <sqlalchemy.engine.base.Connection at 0x7f4b1a315b70>)

In [3]:
folder_url = 's3://production-data-science-nextail.co/seasonality_data/client=pimkie/execution_id=4c668958-acbb-4f91-bbfa-8644cdfa55d4'

In [4]:
STORE_PRODUCT_CLUSTERS = 'store_product_clusters.csv'
SEASONALITY_STORE_PRODUCT_CLUSTERS = 'seasonality_store_product_clusters.csv'
STORE_WEEKDAY_WEIGHTS = 'store_weekday_weights.csv'

In [5]:
store_product_clusters = read_df_from_csv_in_s3_url(f'{folder_url}/{STORE_PRODUCT_CLUSTERS}')
seasonality_store_product_clusters = read_df_from_csv_in_s3_url(f'{folder_url}/{SEASONALITY_STORE_PRODUCT_CLUSTERS}')
store_weekday_weights = read_df_from_csv_in_s3_url(f'{folder_url}/{STORE_WEEKDAY_WEIGHTS}')

In [6]:
today = datetime.date.today().strftime('%Y-%m-%d')
print(type(today))

<class 'str'>


# - Repositories for sales and stock

In [7]:
sales_repo = SalesRepository(request_data)

In [8]:
stock_repo = StocksRepository(request_data)

# - Create a product_sku dataframe and a feature dataframe

In [9]:
sql = f'''
        WITH domain as (
        SELECT DISTINCT products.switch_product_id AS switch_product_id
        FROM products
        WHERE TRUE
        )
        SELECT DISTINCT switch_product_id as product_id, skus.switch_sku_id as sku_id, families.name as family, 
        seasons.name as season, products.full_price as price
        FROM domain
        JOIN products USING(switch_product_id)
        JOIN skus ON skus.product_id = products.id
        JOIN families ON families.id = products.family_id
        JOIN seasons ON seasons.id = products.season_id
        WHERE products.family_id = 8
        ORDER BY product_id
        '''

sql_data = sql_to_pandas_big(sql, request_data.redshift_con)
features_products_2 = sql_data[['product_id', 'family', 'season', 'price']].drop_duplicates()
product_sku = sql_data[['product_id', 'sku_id']]

In [10]:
features_products_2.head(3)

,product_id,family,season,price
0,2134,T-Shirt,2018S,4.0
1,2135,T-Shirt,2018S,4.0
2,2145,T-Shirt,2018S,6.0


In [11]:
features_products_2['price'] = round(features_products_2['price'])
features_products_2.reset_index(inplace=True, drop=True)
features_products_2['year_season'] = [elem[0:4] if type(elem)==str else elem for elem in features_products_2['season']]
features_products_2['season'] = [elem[-1] if type(elem)==str else elem for elem in features_products_2['season']]
features_products_2['season'].replace({'S':'summer', 'W':'winter'}, inplace=True)
features_products_2.head()

,product_id,family,season,price,year_season
0,2134,T-Shirt,summer,4.0,2018
1,2135,T-Shirt,summer,4.0,2018
2,2145,T-Shirt,summer,6.0,2018
3,2146,T-Shirt,summer,6.0,2018
4,2150,T-Shirt,summer,6.0,2018


In [12]:
features_products_2 = pd.get_dummies(data=features_products_2, columns = ['family', 'season'])
features_products = features_products_2.drop(columns={'year_season'}, axis=1)
features_products.head()

,product_id,price,family_T-Shirt,season_summer,season_winter
0,2134,4.0,1,1,0
1,2135,4.0,1,1,0
2,2145,6.0,1,1,0
3,2146,6.0,1,1,0
4,2150,6.0,1,1,0


In [13]:
#features_products.to_csv('new_data/features_products.csv')

In [14]:
product_sku.head()

,product_id,sku_id
0,2134,16060
1,2135,16070
2,2145,16140
3,2146,16144
4,2146,16143


In [15]:
product_n_sku = product_sku.groupby('product_id').agg({'sku_id': [pd.Series.nunique]})
product_n_sku.columns = product_n_sku.columns.droplevel(1)
product_n_sku.rename({'sku_id': 'skus'}, axis=1, inplace=True)
product_n_sku.reset_index(drop=False, inplace=True)
product_n_sku.head()

,product_id,skus
0,2134,5
1,2135,6
2,2136,4
3,2137,6
4,2138,4


### - List of product_ids and sku_ids

In [16]:
product_ids = tuple(product_sku['product_id'].unique())
sku_ids = tuple(product_sku['sku_id'].unique())

In [17]:
len(product_ids)

3034

# - Sales and stock of product_ids

- sales

In [113]:
sales_all = sales_repo.get_sales(min_date='2010-01-01', max_date=today, ids=product_ids, with_returns=True, include_linked_lines=True)
sales_all.reset_index(drop=True, inplace=True)
sales_all.head()

,date,store_id,product_id,quantity
0,2018-04-06,514,2899,1
1,2018-11-26,805,3347,-1
2,2019-05-10,147,3694,1
3,2020-01-25,574,4659,1
4,2020-12-05,131,12332,1


In [130]:
sales_tshirts = sales_all[sales_all['product_id'].isin(product_ids)].reset_index(drop=True)
sales_tshirts.head()

,date,store_id,product_id,quantity
0,2018-04-06,514,2899,1
1,2018-11-26,805,3347,-1
2,2019-05-10,147,3694,1
3,2020-01-25,574,4659,1
4,2020-12-05,131,12332,1


In [131]:
sales_tshirts.product_id.nunique()

2813

In [132]:
sales_tshirts['first_sale'] = sales_tshirts.groupby('product_id')['date'].transform('min')
sales_tshirts['one_year_from_first_sale'] = sales_tshirts['first_sale'] + pd.offsets.DateOffset(years=1)

sales_tshirts = sales_tshirts[(sales_tshirts['date']>=sales_tshirts['first_sale']) & 
                              (sales_tshirts['first_sale']<='2021-3-15') & ### filter those new items
                              (sales_tshirts['date']<=sales_tshirts['one_year_from_first_sale'])]

sales_tshirts = sales_tshirts[~((sales_tshirts['date']>='2020-3-1')&(sales_tshirts['date']<='2020-9-1'))]

sales_tshirts = sales_tshirts[~((sales_tshirts['first_sale']>='2019-12-1')&(sales_tshirts['first_sale']<='2020-9-1'))]

# sales_tshirts.drop(['one_year_from_first_sale', 'first_sale'], axis=1, inplace=True)

sales_tshirts['month'] = sales_tshirts['date'].dt.month
sales_tshirts['year'] = sales_tshirts['date'].dt.year

sales_tshirts.sort_values(['product_id', 'store_id', 'date'], inplace=True)
sales_tshirts.reset_index(drop=True, inplace=True)


sales_tshirts.head()

,date,store_id,product_id,quantity,first_sale,one_year_from_first_sale,month,year
0,2018-02-17,1,2134,1,2018-02-13,2019-02-13,2,2018
1,2018-02-20,1,2134,3,2018-02-13,2019-02-13,2,2018
2,2018-02-21,1,2134,1,2018-02-13,2019-02-13,2,2018
3,2018-03-01,1,2134,1,2018-02-13,2019-02-13,3,2018
4,2018-03-05,1,2134,2,2018-02-13,2019-02-13,3,2018


In [133]:
sales_tshirts.product_id.nunique()

2183

In [144]:
display(sales_tshirts[sales_tshirts['first_sale']=='2019-12-25'])

,date,store_id,product_id,quantity,first_sale,one_year_from_first_sale,month,year


In [145]:
sales_tshirts.drop(['one_year_from_first_sale', 'first_sale'], axis=1, inplace=True)

In [146]:
sales_tshirts['weekday'] = sales_tshirts['date'].dt.dayofweek

In [147]:
sales_tshirts.head()

,date,store_id,product_id,quantity,month,year,weekday
0,2018-02-17,1,2134,1,2,2018,5
1,2018-02-20,1,2134,3,2,2018,1
2,2018-02-21,1,2134,1,2,2018,2
3,2018-03-01,1,2134,1,3,2018,3
4,2018-03-05,1,2134,2,3,2018,0


In [148]:
store_product_clusters.head()

,cluster_id,store_id,product_id
0,1,1,1
1,1,1,2
2,1,1,3
3,1,1,4
4,1,1,5


# - Seasonality

In [26]:
seasonality_store_product_clusters.head()

,date,cluster_id,coefficient
0,2017-12-10,1,1.634459
1,2017-12-11,1,0.831088
2,2017-12-12,1,0.890900
3,2017-12-13,1,0.779568
4,2017-12-14,1,0.732426


In [27]:
# Moving average
seasonality_store_coeff = seasonality_store_product_clusters.copy(deep = True)
# display(seasonality_store_coeff)
rolling_k = 3
seasonality_store_coeff['moving_ave_coeff'] = seasonality_store_coeff.groupby('cluster_id',as_index = False)['coefficient'].rolling(rolling_k).mean().values
# display(moving_ave_coeff)
# seasonality_store_coeff = seasonality_store_coeff.merge(moving_ave_coeff,on = ['date','cluster_id'])

display(seasonality_store_coeff.head())

,date,cluster_id,coefficient,moving_ave_coeff
0,2017-12-10,1,1.634459,NaN
1,2017-12-11,1,0.831088,NaN
2,2017-12-12,1,0.890900,1.118815
3,2017-12-13,1,0.779568,0.833852
4,2017-12-14,1,0.732426,0.800964


In [53]:
##fill nan values
# for i in range(len(seasonality_store_coeff)):
#     if seasonality_store_coeff.iloc[i,:]['moving_ave_coeff'] == np.nan:
#         seasonality_store_coeff.iloc[i,:]['moving_ave_coeff'] = seasonality_store_coeff.iloc[i,:]['coefficient']
seasonality_store_coeff = seasonality_store_coeff.fillna(axis = 1, method = 'ffill')


In [54]:
seasonality_store_coeff['date'] = pd.to_datetime(seasonality_store_coeff['date'], format='%Y-%m-%d')
seasonality_store_coeff = seasonality_store_coeff.convert_objects(convert_numeric = True)
display(seasonality_store_coeff)
display(seasonality_store_coeff.dtypes)

,date,cluster_id,coefficient,moving_ave_coeff
0,2017-12-10,1,1.634459,1.634459
1,2017-12-11,1,0.831088,0.831088
2,2017-12-12,1,0.890900,1.118815
3,2017-12-13,1,0.779568,0.833852
4,2017-12-14,1,0.732426,0.800964
5,2017-12-15,1,0.705994,0.739329
6,2017-12-16,1,0.512248,0.650222
7,2017-12-17,1,5.013565,2.077269
8,2017-12-18,1,0.859995,2.128603
9,2017-12-19,1,0.921709,2.265090


date                datetime64[ns]
cluster_id                   int64
coefficient                float64
moving_ave_coeff           float64
dtype: object

In [55]:
display(seasonality_store_coeff[seasonality_store_coeff['date']=='2018-02-17'])

,date,cluster_id,coefficient,moving_ave_coeff
69,2018-02-17,1,0.710108,1.019466
2105,2018-02-17,2,0.810409,1.018742
4141,2018-02-17,3,0.846089,1.053950
6177,2018-02-17,4,0.738107,1.047487
8213,2018-02-17,5,0.785134,1.068267
10249,2018-02-17,6,0.800473,1.041375
12285,2018-02-17,7,0.813447,0.928760
14321,2018-02-17,8,0.957632,1.071595
16357,2018-02-17,9,0.788874,1.114587
18393,2018-02-17,10,1.000000,1.000000


In [56]:
store_weekday_weights.head()

,store_id,weekday,weight
0,1,0,0.830237
1,1,1,0.778988
2,1,2,1.072389
3,1,3,1.060858
4,1,4,1.100577


In [57]:
## Moving average
## !!!!!!!!!!!!!!!!This weight may not need moving average
store_weekday_weights_ave = store_weekday_weights.copy(deep = True)
store_weekday_weights_ave['moving_ave_coefficients'] = store_weekday_weights_ave.groupby('store_id',as_index = False)['weight'].rolling(rolling_k).mean().values

store_weekday_weights_ave = store_weekday_weights_ave.fillna(axis = 1, method = 'ffill')
display(store_weekday_weights_ave)

,store_id,weekday,weight,moving_ave_coefficients
0,1.0,0.0,0.830237,0.830237
1,1.0,1.0,0.778988,0.778988
2,1.0,2.0,1.072389,0.893871
3,1.0,3.0,1.060858,0.970745
4,1.0,4.0,1.100577,1.077941
5,1.0,5.0,2.102498,1.421311
6,1.0,6.0,0.054452,1.085842
7,2.0,0.0,0.894121,0.894121
8,2.0,1.0,0.825638,0.825638
9,2.0,2.0,0.979885,0.899881


## Merge coeff with sales

In [149]:
sales_tshirts_ = sales_tshirts.merge(store_weekday_weights, on=['store_id', 'weekday'], how='left')
sales_tshirts_.head()

,date,store_id,product_id,quantity,month,year,weekday,weight
0,2018-02-17,1,2134,1,2,2018,5,2.102498
1,2018-02-20,1,2134,3,2,2018,1,0.778988
2,2018-02-21,1,2134,1,2,2018,2,1.072389
3,2018-03-01,1,2134,1,3,2018,3,1.060858
4,2018-03-05,1,2134,2,3,2018,0,0.830237


In [150]:
sales_tshirts_ = sales_tshirts_.merge(store_product_clusters, on=['store_id', 'product_id'], how='left')
sales_tshirts_.head()

,date,store_id,product_id,quantity,month,year,weekday,weight,cluster_id
0,2018-02-17,1,2134,1,2,2018,5,2.102498,1
1,2018-02-20,1,2134,3,2,2018,1,0.778988,1
2,2018-02-21,1,2134,1,2,2018,2,1.072389,1
3,2018-03-01,1,2134,1,3,2018,3,1.060858,1
4,2018-03-05,1,2134,2,3,2018,0,0.830237,1


In [151]:
seasonality_store_coeff.dtypes

date                datetime64[ns]
cluster_id                   int64
coefficient                float64
moving_ave_coeff           float64
dtype: object

In [152]:
seasonality_store_coeff['cluster_id'] = seasonality_store_coeff['cluster_id'].apply(int)
# seasonality_store_coeff['coefficient'] = seasonality_store_coeff['coefficient'].apply('float64')
# seasonality_store_coeff['moving_ave_coeff'] = seasonality_store_coeff['cluster_id'].apply('float64')

display(seasonality_store_coeff.dtypes)
sales_tshirts_ = sales_tshirts_.merge(seasonality_store_coeff, on=['date', 'cluster_id'], how='left')
sales_tshirts_.head()

date                datetime64[ns]
cluster_id                   int64
coefficient                float64
moving_ave_coeff           float64
dtype: object

,date,store_id,product_id,quantity,month,year,weekday,weight,cluster_id,coefficient,moving_ave_coeff
0,2018-02-17,1,2134,1,2,2018,5,2.102498,1,0.710108,1.019466
1,2018-02-20,1,2134,3,2,2018,1,0.778988,1,1.106564,0.774550
2,2018-02-21,1,2134,1,2,2018,2,1.072389,1,1.327876,1.155960
3,2018-03-01,1,2134,1,3,2018,3,1.060858,1,1.214111,1.227733
4,2018-03-05,1,2134,2,3,2018,0,0.830237,1,1.082823,0.662506


In [153]:
sales_tshirts_.isna().any()

date                False
store_id            False
product_id          False
quantity            False
month               False
year                False
weekday             False
weight              False
cluster_id          False
coefficient         False
moving_ave_coeff    False
dtype: bool

In [154]:
sales_tshirts_['total_coeff'] = sales_tshirts_['weight']*sales_tshirts_['moving_ave_coeff']
display(sales_tshirts_.head())
print('min weight', sales_tshirts_['weight'].min())
print('max weight', sales_tshirts_['weight'].max())
print('min coefficient', sales_tshirts_['moving_ave_coeff'].min())
print('max coefficient', sales_tshirts_['moving_ave_coeff'].max())

# sales_tshirts_.to_csv('test.csv')

,date,store_id,product_id,quantity,month,year,weekday,weight,cluster_id,coefficient,moving_ave_coeff,total_coeff
0,2018-02-17,1,2134,1,2,2018,5,2.102498,1,0.710108,1.019466,2.143426
1,2018-02-20,1,2134,3,2,2018,1,0.778988,1,1.106564,0.774550,0.603365
2,2018-02-21,1,2134,1,2,2018,2,1.072389,1,1.327876,1.155960,1.239639
3,2018-03-01,1,2134,1,3,2018,3,1.060858,1,1.214111,1.227733,1.302451
4,2018-03-05,1,2134,2,3,2018,0,0.830237,1,1.082823,0.662506,0.550037


min weight 0.0
max weight 2.7096774193548385
min coefficient 0.0
max coefficient 53.53291570274924


### deal with unusual coefficient

In [155]:
## find the index of 0 total coeff value
print(len(sales_tshirts_))
# display(sales_tshirts_.iloc[sales_tshirts_['total_coeff'].idxmin()])
print('how many 0 in weight: ',len(sales_tshirts_[sales_tshirts_['weight']==0]))
sales_tshirts_.loc[sales_tshirts_['weight'] <= 0.1,'weight'] = 1
# display(sales_tshirts_.iloc[sales_tshirts_['total_coeff'].idxmin()])
sales_tshirts_.loc[sales_tshirts_['moving_ave_coeff'] <= 0.1,'moving_ave_coeff'] = 1
sales_tshirts_['total_coeff'] = sales_tshirts_['weight']*sales_tshirts_['moving_ave_coeff']

14084144
how many 0 in weight:  25345


In [156]:
display(sales_tshirts_.head())
sales_tshirts_.loc[sales_tshirts_['total_coeff'] <= 0.1,'total_coeff'] = 0.1
print('min total coefficient', sales_tshirts_['total_coeff'].min())
print('max total coefficient sale', sales_tshirts_['total_coeff'].max())

,date,store_id,product_id,quantity,month,year,weekday,weight,cluster_id,coefficient,moving_ave_coeff,total_coeff
0,2018-02-17,1,2134,1,2,2018,5,2.102498,1,0.710108,1.019466,2.143426
1,2018-02-20,1,2134,3,2,2018,1,0.778988,1,1.106564,0.774550,0.603365
2,2018-02-21,1,2134,1,2,2018,2,1.072389,1,1.327876,1.155960,1.239639
3,2018-03-01,1,2134,1,3,2018,3,1.060858,1,1.214111,1.227733,1.302451
4,2018-03-05,1,2134,2,3,2018,0,0.830237,1,1.082823,0.662506,0.550037


min total coefficient 0.1
max total coefficient sale 53.53291570274924


In [157]:
print('how may negative value in weight: ', len(sales_tshirts_[sales_tshirts_['weight']<0]))
print('how may negative value in seasonality: ', len(sales_tshirts_[sales_tshirts_['coefficient']<0]))
print('how may negative value in quantity: ', len(sales_tshirts_[sales_tshirts_['quantity']<0]))

how may negative value in weight:  0
how may negative value in seasonality:  0
how may negative value in quantity:  282962


### get revised quantity sale

In [158]:
sales_tshirts_['quantity_seasonality'] = sales_tshirts_['quantity'] / sales_tshirts_['total_coeff']

## revise negative(return) seasionality quantity
sales_tshirts_.loc[sales_tshirts_['quantity_seasonality'] < 0,'quantity_seasonality'] = sales_tshirts_.loc[sales_tshirts_['quantity_seasonality'] < 0,'quantity']
display(sales_tshirts_.head())
print('min quantity_seasonality', sales_tshirts_['quantity_seasonality'].min())
print('max quantity_seasonality', sales_tshirts_['quantity_seasonality'].max())
display(sales_tshirts_.loc[sales_tshirts_['quantity_seasonality'].idxmax()])
display(sales_tshirts_.loc[sales_tshirts_['quantity_seasonality'] < 0].head())

,date,store_id,product_id,quantity,month,year,weekday,weight,cluster_id,coefficient,moving_ave_coeff,total_coeff,quantity_seasonality
0,2018-02-17,1,2134,1,2,2018,5,2.102498,1,0.710108,1.019466,2.143426,0.466543
1,2018-02-20,1,2134,3,2,2018,1,0.778988,1,1.106564,0.774550,0.603365,4.972112
2,2018-02-21,1,2134,1,2,2018,2,1.072389,1,1.327876,1.155960,1.239639,0.806686
3,2018-03-01,1,2134,1,3,2018,3,1.060858,1,1.214111,1.227733,1.302451,0.767783
4,2018-03-05,1,2134,2,3,2018,0,0.830237,1,1.082823,0.662506,0.550037,3.636120


min quantity_seasonality -25.0
max quantity_seasonality 714.1863905538069


date                    2019-01-15 00:00:00
store_id                                707
product_id                             3593
quantity                                923
month                                     1
year                                   2019
weekday                                   1
weight                              1.29238
cluster_id                              707
coefficient                               1
moving_ave_coeff                          1
total_coeff                         1.29238
quantity_seasonality                714.186
Name: 6118865, dtype: object

,date,store_id,product_id,quantity,month,year,weekday,weight,cluster_id,coefficient,moving_ave_coeff,total_coeff,quantity_seasonality
110,2018-03-12,6,2134,-1,3,2018,0,1.018115,6,0.915417,0.622647,0.633926,-1.0
174,2018-02-24,9,2134,-1,2,2018,5,2.073842,9,0.797554,1.126688,2.336572,-1.0
223,2018-05-22,11,2134,-1,5,2018,1,0.678348,11,1.577362,1.305771,0.885767,-1.0
241,2018-03-30,12,2134,-2,3,2018,4,1.106675,12,1.052703,1.157726,1.281227,-2.0
283,2018-03-30,14,2134,-1,3,2018,4,1.096339,14,1.045297,1.089579,1.194547,-1.0


In [159]:
sales_tshirts_.dtypes

date                    datetime64[ns]
store_id                         int32
product_id                       int32
quantity                         int16
month                            int64
year                             int64
weekday                          int64
weight                         float64
cluster_id                       int64
coefficient                    float64
moving_ave_coeff               float64
total_coeff                    float64
quantity_seasonality           float64
dtype: object

In [160]:
sales_tshirts.columns

Index(['date', 'store_id', 'product_id', 'quantity', 'month', 'year',
       'weekday'],
      dtype='object')

In [161]:

sales_tshirts = sales_tshirts.merge(sales_tshirts_[['date', 'store_id', 'product_id', 'quantity', 'month', 'year', 'weekday', 'quantity_seasonality']], 
                                    on=['date', 'store_id', 'product_id', 'quantity', 'month', 'year', 'weekday'], how='left')
sales_tshirts.head()


,date,store_id,product_id,quantity,month,year,weekday,quantity_seasonality
0,2018-02-17,1,2134,1,2,2018,5,0.466543
1,2018-02-20,1,2134,3,2,2018,1,4.972112
2,2018-02-21,1,2134,1,2,2018,2,0.806686
3,2018-03-01,1,2134,1,3,2018,3,0.767783
4,2018-03-05,1,2134,2,3,2018,0,3.636120


In [162]:
if 'quantity_seasonality_x' in list(sales_tshirts):
    sales_tshirts = sales_tshirts.drop(['quantity_seasonality_x'],axis = 1)
    
#     
sales_tshirts = sales_tshirts.rename(columns = {'quantity_seasonality_y':'quantity_seasonality'})
display(sales_tshirts.head())

,date,store_id,product_id,quantity,month,year,weekday,quantity_seasonality
0,2018-02-17,1,2134,1,2,2018,5,0.466543
1,2018-02-20,1,2134,3,2,2018,1,4.972112
2,2018-02-21,1,2134,1,2,2018,2,0.806686
3,2018-03-01,1,2134,1,3,2018,3,0.767783
4,2018-03-05,1,2134,2,3,2018,0,3.636120


- Sales Tshirts

In [163]:
quantity = 'quantity_seasonality'
sales_tshirts_agg = sales_tshirts.groupby(['product_id', 'store_id', 'month', 'year']).agg({quantity: ['sum']})

sales_tshirts_agg.columns = sales_tshirts_agg.columns.droplevel(1)
sales_tshirts_agg.reset_index(inplace=True, drop=False)
sales_tshirts_agg.rename(columns={quantity: 'quantity_sales'}, inplace=True)

sales_tshirts_agg.head()

,product_id,store_id,month,year,quantity_sales
0,2134,1,2,2018,6.245342
1,2134,1,3,2018,12.778248
2,2134,1,4,2018,12.651702
3,2134,1,5,2018,1.255968
4,2134,2,2,2018,0.983885


# Cut below

- stock

In [149]:
stock_2017 = stock_repo.get_stocks(min_date='2017-01-01', max_date='2017-12-31', ids=sku_ids, by_sku=True)
stock_2017.reset_index(drop=True, inplace=True)

In [150]:
stock_2018_1 = stock_repo.get_stocks(min_date='2018-01-01', max_date='2018-06-30', ids=sku_ids, by_sku=True)
stock_2018_2 = stock_repo.get_stocks(min_date='2018-07-01', max_date='2018-12-31', ids=sku_ids, by_sku=True)
stock_2018_1.reset_index(drop=True, inplace=True)
stock_2018_2.reset_index(drop=True, inplace=True)

In [151]:
stock_2019_1 = stock_repo.get_stocks(min_date='2019-01-01', max_date='2019-06-30', ids=sku_ids, by_sku=True)
stock_2019_2 = stock_repo.get_stocks(min_date='2019-07-01', max_date='2019-09-30', ids=sku_ids, by_sku=True)
stock_2019_3 = stock_repo.get_stocks(min_date='2019-10-01', max_date='2019-12-31', ids=sku_ids, by_sku=True)
stock_2019_1.reset_index(drop=True, inplace=True)
stock_2019_2.reset_index(drop=True, inplace=True)
stock_2019_3.reset_index(drop=True, inplace=True)

In [152]:
stock_2021 = stock_repo.get_stocks(min_date='2021-01-01', max_date=today, ids=sku_ids, by_sku=True)
stock_2021.reset_index(drop=True, inplace=True)
stock_2021.head()

,date,store_id,sku_id,stock_commercial
0,2021-07-13,766,35778,2
1,2021-03-27,357,79843,3
2,2021-03-01,918,78584,1
3,2021-05-23,910,77775,1
4,2021-03-21,489,72835,1


In [153]:
product_sku['sku_id'] = product_sku['sku_id'].astype(int)
stock_2017 = stock_2017.merge(product_sku, on='sku_id', how='left')
stock_2018_1 = stock_2018_1.merge(product_sku, on='sku_id', how='left')
stock_2018_2 = stock_2018_2.merge(product_sku, on='sku_id', how='left')
stock_2019_1 = stock_2019_1.merge(product_sku, on='sku_id', how='left')
stock_2019_2 = stock_2019_2.merge(product_sku, on='sku_id', how='left')
stock_2019_3 = stock_2019_3.merge(product_sku, on='sku_id', how='left')
stock_2021 = stock_2021.merge(product_sku, on='sku_id', how='left')
stock_2017.head()

,date,store_id,sku_id,stock_commercial,product_id
0,2017-12-17,161,16646,1,2218
1,2017-12-17,458,16646,1,2218
2,2017-12-17,541,16646,1,2218
3,2017-12-17,615,16646,3,2218
4,2017-12-17,691,16646,1,2218


In [154]:
stock_2017['month'] = stock_2017['date'].dt.month
stock_2018_1['month'] = stock_2018_1['date'].dt.month
stock_2018_2['month'] = stock_2018_2['date'].dt.month
stock_2019_1['month'] = stock_2019_1['date'].dt.month
stock_2019_2['month'] = stock_2019_2['date'].dt.month
stock_2019_3['month'] = stock_2019_3['date'].dt.month
stock_2021['month'] = stock_2021['date'].dt.month

stock_2017['year'] = stock_2017['date'].dt.year
stock_2018_1['year'] = stock_2018_1['date'].dt.year
stock_2018_2['year'] = stock_2018_2['date'].dt.year
stock_2019_1['year'] = stock_2019_1['date'].dt.year
stock_2019_2['year'] = stock_2019_2['date'].dt.year
stock_2019_3['year'] = stock_2019_3['date'].dt.year
stock_2021['year'] = stock_2021['date'].dt.year
stock_2019_2.head(3)

,date,store_id,sku_id,stock_commercial,product_id,month,year
0,2019-07-07,615,16079,1,2137,7,2019
1,2019-07-07,751,16079,5,2137,7,2019
2,2019-07-07,779,16079,2,2137,7,2019


- Stocks Tshirts

In [155]:
stock_tshirts_agg_2017 = stock_2017.groupby(['product_id', 'store_id', 'month', 'year']).agg({'sku_id': [pd.Series.nunique], 'stock_commercial': ['sum']})

In [156]:
stock_tshirts_agg_2018_1 = stock_2018_1.groupby(['product_id', 'store_id', 'month', 'year']).agg({'sku_id': [pd.Series.nunique], 'stock_commercial': ['sum']})


In [157]:
stock_tshirts_agg_2018_2 = stock_2018_2.groupby(['product_id', 'store_id', 'month', 'year']).agg({'sku_id': [pd.Series.nunique], 'stock_commercial': ['sum']})


In [158]:
stock_tshirts_agg_2019_1 = stock_2019_1.groupby(['product_id', 'store_id', 'month', 'year']).agg({'sku_id': [pd.Series.nunique], 'stock_commercial': ['sum']})


In [159]:
stock_tshirts_agg_2019_2 = stock_2019_2.groupby(['product_id', 'store_id', 'month', 'year']).agg({'sku_id': [pd.Series.nunique], 'stock_commercial': ['sum']})


KeyboardInterrupt: 

In [ ]:
stock_tshirts_agg_2019_3 = stock_2019_3.groupby(['product_id', 'store_id', 'month', 'year']).agg({'sku_id': [pd.Series.nunique], 'stock_commercial': ['sum']})


In [ ]:
stock_tshirts_agg_2021 = stock_2021.groupby(['product_id', 'store_id', 'month', 'year']).agg({'sku_id': [pd.Series.nunique], 'stock_commercial': ['sum']})


In [ ]:
stock_tshirts_agg_2017.reset_index(drop=False, inplace=True)
stock_tshirts_agg_2018_1.reset_index(drop=False, inplace=True)
stock_tshirts_agg_2018_2.reset_index(drop=False, inplace=True)
stock_tshirts_agg_2019_1.reset_index(drop=False, inplace=True)
stock_tshirts_agg_2019_2.reset_index(drop=False, inplace=True)
stock_tshirts_agg_2019_3.reset_index(drop=False, inplace=True)
stock_tshirts_agg_2021.reset_index(drop=False, inplace=True)

In [ ]:
frames = [stock_tshirts_agg_2017,stock_tshirts_agg_2018_1,stock_tshirts_agg_2018_2,stock_tshirts_agg_2019_1,stock_tshirts_agg_2019_2,stock_tshirts_agg_2019_3, 
          stock_tshirts_agg_2021]

stock_tshirts_agg = pd.concat(frames)
stock_tshirts_agg.columns = stock_tshirts_agg.columns.droplevel(1)
stock_tshirts_agg.rename(columns={'sku_id': 'sku_stock'}, inplace=True)
stock_tshirts_agg.sort_values(['product_id', 'year', 'month'], inplace=True)
stock_tshirts_agg.head(3)

In [ ]:
stock_tshirts_agg = stock_tshirts_agg.merge(product_n_sku, on='product_id', how='left')
stock_tshirts_agg.head()

- Stock

In [ ]:
"""stock_tshirts_agg = stock_tshirts_agg.merge(features_products_2, on='product_id', how='left')
stock_tshirts_agg['year_season'] = stock_tshirts_agg['year_season'].astype(int)
stock_tshirts_agg_sku = stock_tshirts_agg[stock_tshirts_agg['year']==stock_tshirts_agg['year_season']]
stock_tshirts_agg.head()"""

In [ ]:
stock_tshirts_agg_sku = stock_tshirts_agg[(stock_tshirts_agg['sku_stock']==stock_tshirts_agg['skus']) | (stock_tshirts_agg['sku_stock']>=3)]
stock_tshirts_agg_sku.head()

In [ ]:
stock_tshirts_agg_sku = stock_tshirts_agg_sku.groupby(['product_id', 'store_id', 'month', 'year']).agg({'stock_commercial': ['sum']})

stock_tshirts_agg_sku.columns = stock_tshirts_agg_sku.columns.droplevel(1)
stock_tshirts_agg_sku.reset_index(inplace=True, drop=False)
stock_tshirts_agg_sku.rename(columns={'stock_commercial': 'stock'}, inplace=True)

stock_tshirts_agg_sku.head()

In [ ]:
len(stock_tshirts_agg_sku), len(sales_tshirts_agg)

In [ ]:
stock_tshirts_agg_sku.product_id.nunique(), sales_tshirts_agg.product_id.nunique()

In [ ]:
stock_tshirts_agg_sku.to_csv('t_stocks.csv')

# continue here

In [164]:
stock_tshirts_agg_sku = pd.read_csv('t_stocks.csv')

- Merge

In [165]:
tshirts = pd.merge(stock_tshirts_agg_sku, sales_tshirts_agg, 
                   on=['product_id', 'store_id', 'month', 'year'],
                  how = 'inner')
tshirts.sort_values(['product_id', 'year', 'month', 'store_id'], inplace=True)
tshirts.reset_index(drop=True, inplace=True)
tshirts.head()

,Unnamed: 0,product_id,store_id,month,year,stock,quantity_sales
0,0,2134,1,2,2018,9,6.245342
1,2,2134,2,2,2018,9,0.983885
2,8,2134,4,2,2018,7,1.247348
3,11,2134,5,2,2018,5,2.119078
4,13,2134,6,2,2018,7,6.590104


# - Potential

In [166]:
tshirts_potential = tshirts.groupby(['product_id', 'year', 'month']).agg({'store_id': pd.Series.nunique, 'stock': 'sum', 'quantity_sales': 'sum'})
tshirts_potential.rename(columns={'store_id': 'stores_count'}, inplace=True)
tshirts_potential.head()

stores_count  stock  quantity_sales
product_id year month                                     
2134       2018 2               401   3206     1118.977183
                3               685  10363     5363.264792
                4               553   8246     3994.226397
                5                87   1124      299.927099
                6                27    384      108.875592

In [167]:
tshirts_potential['potential'] = tshirts_potential['quantity_sales']/tshirts_potential['stores_count']
#tshirts_potential.reset_index(drop=False, inplace=True)
tshirts_potential.head(10)
#tshirts_potential.to_csv('new_data/tshirt_potential_per_month.csv')

stores_count  stock  quantity_sales  potential
product_id year month                                                
2134       2018 2               401   3206     1118.977183   2.790467
                3               685  10363     5363.264792   7.829584
                4               553   8246     3994.226397   7.222833
                5                87   1124      299.927099   3.447438
                6                27    384      108.875592   4.032429
                7                20    184      104.870200   5.243510
                8                 1      7        3.695017   3.695017
2135       2018 2               383   3284     1646.670137   4.299400
                3               676  12014     8727.297533  12.910203
                4               655  12480     8952.673775  13.668204

# - Potentials per month

In [168]:
tshirts_potential_month = tshirts_potential.drop(columns={'stores_count', 'stock', 'quantity_sales'}, axis=1)
tshirts_potential_month.head(3)

potential
product_id year month           
2134       2018 2       2.790467
                3       7.829584
                4       7.222833

In [203]:
tshirts_potential_month = tshirts_potential_month.potential.unstack().reset_index()
tshirts_potential_month.head()

month,product_id,year,1,2,3,4,5,6,7,8,9,10,11,12
0,2134,2018,NaN,2.790467,7.829584,7.222833,2.717109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2135,2018,NaN,4.299400,12.910203,13.668204,7.505080,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2136,2018,NaN,3.134337,9.353269,10.970604,6.278878,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2137,2018,1.397129,1.837797,3.176483,2.970588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2138,2018,1.232091,1.898363,3.154236,1.904066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [204]:
tshirts_potential_month = pd.get_dummies(data=tshirts_potential_month, columns = ['year'])
tshirts_potential_month = tshirts_potential_month.groupby('product_id').sum()
tshirts_potential_month.reset_index(inplace=True)
tshirts_potential_month.head(3)

,product_id,1,2,3,4,5,6,7,8,9,10,11,12,year_2017,year_2018,year_2019,year_2021
0,2134,0.0,2.790467,7.829584,7.222833,2.717109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
1,2135,0.0,4.299400,12.910203,13.668204,7.505080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
2,2136,0.0,3.134337,9.353269,10.970604,6.278878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0


In [205]:
#tshirts_potential_month.to_csv('new_data/tshirts_potential_month_seasonality.csv')

# - Potential calculations

In [169]:
tshirts_potential.reset_index(drop=False, inplace=True)
tshirts_potential.head()

,product_id,year,month,stores_count,stock,quantity_sales,potential
0,2134,2018,2,401,3206,1118.977183,2.790467
1,2134,2018,3,685,10363,5363.264792,7.829584
2,2134,2018,4,553,8246,3994.226397,7.222833
3,2134,2018,5,87,1124,299.927099,3.447438
4,2134,2018,6,27,384,108.875592,4.032429


In [170]:
tshirts_potential.product_id.nunique()

2049

In [171]:
tshirts_potential['potential_mean'] = tshirts_potential.groupby(['product_id'])['potential'].transform('mean')
tshirts_potential['potential_std'] = tshirts_potential.groupby(['product_id'])['potential'].transform('std')

tshirts_potential['potential_025'] = tshirts_potential['potential_mean'] - 0.25*tshirts_potential['potential_std']
#tshirts_potential['quantity_sales_05'] = tshirts_potential['quantity_sales_mean'] - 0.5*tshirts_potential['quantity_sales_std']

tshirts_potential.drop(['potential_std'], axis=1, inplace=True)

tshirts_potential.reset_index(drop=True, inplace=True)

tshirts_potential.head()

,product_id,year,month,stores_count,stock,quantity_sales,potential,potential_mean,potential_025
0,2134,2018,2,401,3206,1118.977183,2.790467,4.894468,4.406482
1,2134,2018,3,685,10363,5363.264792,7.829584,4.894468,4.406482
2,2134,2018,4,553,8246,3994.226397,7.222833,4.894468,4.406482
3,2134,2018,5,87,1124,299.927099,3.447438,4.894468,4.406482
4,2134,2018,6,27,384,108.875592,4.032429,4.894468,4.406482


In [172]:
tshirts_potential_025 = tshirts_potential[tshirts_potential['potential']>=tshirts_potential['potential_025']]
tshirts_potential_025 = tshirts_potential_025[['product_id', 'month', 'year', 'stores_count', 'quantity_sales', 'potential']]
tshirts_potential_025.reset_index(drop=True, inplace=True)
display(tshirts_potential_025.head())

# """tshirts_potential_05 = tshirts_potential[tshirts_potential['quantity_sales']>=tshirts_potential['quantity_sales_05']]
# tshirts_potential_05 = tshirts_potential_05[['product_id', 'month', 'year', 'stores_count', 'quantity_sales', 'potential']]
# tshirts_potential_05.reset_index(drop=True, inplace=True)

# tshirts_potential_mean = tshirts_potential[tshirts_potential['quantity_sales']>=tshirts_potential['quantity_sales_mean']]
# tshirts_potential_mean = tshirts_potential_mean[['product_id', 'month', 'year', 'stores_count', 'quantity_sales', 'potential']]
# tshirts_potential_mean.reset_index(drop=True, inplace=True)"""

,product_id,month,year,stores_count,quantity_sales,potential
0,2134,3,2018,685,5363.264792,7.829584
1,2134,4,2018,553,3994.226397,7.222833
2,2134,7,2018,20,104.870200,5.243510
3,2135,3,2018,676,8727.297533,12.910203
4,2135,4,2018,655,8952.673775,13.668204


In [173]:
display(tshirts_potential_025)
# display(tshirts_potential_025.groupby('product_id')['potential'])

,product_id,month,year,stores_count,quantity_sales,potential
0,2134,3,2018,685,5363.264792,7.829584
1,2134,4,2018,553,3994.226397,7.222833
2,2134,7,2018,20,104.870200,5.243510
3,2135,3,2018,676,8727.297533,12.910203
4,2135,4,2018,655,8952.673775,13.668204
5,2135,5,2018,673,11940.627792,17.742389
6,2135,6,2018,631,8573.712491,13.587500
7,2135,7,2018,232,2482.957386,10.702403
8,2136,3,2018,674,6304.103625,9.353269
9,2136,4,2018,649,7119.921765,10.970604


## second filter!

In [174]:
tshirts_potential_025['potential_mean'] = tshirts_potential_025.groupby(['product_id'])['potential'].transform('mean')
tshirts_potential_025['potential_std'] = tshirts_potential_025.groupby(['product_id'])['potential'].transform('std')

tshirts_potential_025['potential_025'] = tshirts_potential_025['potential_mean'] - 0.25*tshirts_potential_025['potential_std']
#tshirts_potential['quantity_sales_05'] = tshirts_potential['quantity_sales_mean'] - 0.5*tshirts_potential['quantity_sales_std']

tshirts_potential_025.drop(['potential_std'], axis=1, inplace=True)

tshirts_potential_025.reset_index(drop=True, inplace=True)

tshirts_potential_025.head()

,product_id,month,year,stores_count,quantity_sales,potential,potential_mean,potential_025
0,2134,3,2018,685,5363.264792,7.829584,6.765309,6.427213
1,2134,4,2018,553,3994.226397,7.222833,6.765309,6.427213
2,2134,7,2018,20,104.870200,5.243510,6.765309,6.427213
3,2135,3,2018,676,8727.297533,12.910203,13.722140,13.085235
4,2135,4,2018,655,8952.673775,13.668204,13.722140,13.085235


In [175]:
tshirts_potential_025_again = tshirts_potential_025[tshirts_potential_025['potential']>=tshirts_potential_025['potential_025']]

In [176]:
tshirts_potential_025.product_id.nunique()

1998

In [177]:
tshirts_potential_025.head()

,product_id,month,year,stores_count,quantity_sales,potential,potential_mean,potential_025
0,2134,3,2018,685,5363.264792,7.829584,6.765309,6.427213
1,2134,4,2018,553,3994.226397,7.222833,6.765309,6.427213
2,2134,7,2018,20,104.870200,5.243510,6.765309,6.427213
3,2135,3,2018,676,8727.297533,12.910203,13.722140,13.085235
4,2135,4,2018,655,8952.673775,13.668204,13.722140,13.085235


In [212]:
# save tshirts_potential_025, tshirts_potential_05, tshirts_potential_mean
# tshirts_potential_025.to_csv('raw_tshirts_potential_moving_ave.csv')

In [178]:
data_025 = tshirts_potential_025_again.groupby(['product_id']).agg({'potential': 'mean'})
data_025.rename({'potential': 'potential_025'}, axis=1, inplace=True)
data_025.reset_index(drop=False, inplace=True)

# """data_05 = tshirts_potential_05.groupby(['product_id']).agg({'potential': 'mean'})
# data_05.rename({'potential': 'potential_05'}, axis=1, inplace=True)
# data_05.reset_index(drop=False, inplace=True)

# data_mean = tshirts_potential_mean.groupby(['product_id']).agg({'potential': 'mean'})
# data_mean.rename({'potential': 'potential_mean'}, axis=1, inplace=True)
# data_mean.reset_index(drop=False, inplace=True)"""

data_025.head()

,product_id,potential_025
0,2134,7.526208
1,2135,14.999364
2,2136,11.946084
3,2137,8.605577
4,2138,6.923380


# Potential table
tables to merge: tshirts_continent, tshirts_potential_month, data_025, data_025_nm

In [179]:

data_potential = data_025.copy(deep = True)
data_potential.head()

,product_id,potential_025
0,2134,7.526208
1,2135,14.999364
2,2136,11.946084
3,2137,8.605577
4,2138,6.923380


In [180]:
list(data_potential.columns[data_potential.isna().any()])

[]

In [181]:
# data_potential[list(data_potential.columns[data_potential.isna().any()])] = data_potential[list(data_potential.columns[data_potential.isna().any()])].fillna(0)


In [182]:
data_potential.to_csv('New_potential.csv',index = False)